# LightGBM with grid search for hyperparameter optimisation

In this section we are going to create a boosting model for our classification problem.
Boosting is an algorithm which set out to answer the question "Can a set of weak learners create a single strong learner?"[1]
It turns out to be very successful in a wide array of applications. [2]

LightGBM, short for light gradient-boosting machine, is a specific boosting framework developed by microsoft and released open source in 2016.
Although less widely used than XGboost, LightGBM has advantages in efficiency and memory consumption. [3]

We originally wished to use XGboost, but due to some of the problems we came across when implementing the model, LightGBM was the better choice.

In [31]:
import numpy as np
import pandas as pd
from datetime import datetime
from numba import jit
import lightgbm as lgbm
from sklearn.impute import SimpleImputer
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from bayes_opt import BayesianOptimization

Here we have our functions to calculate the gini coefficient, and implement the data handling code to sort out missing values / drop the columns which are mostly missing values and also the calc columns since our EDA discovered these had no correlation to the target. Furthermore, we encode our catagorical features and rescale the data.

In [32]:
def timer(start_time=None):
    if not start_time:
        start_time = datetime.now()
        return start_time
    elif start_time:
        thour, temp_sec = divmod((datetime.now() - start_time).total_seconds(), 3600)
        tmin, tsec = divmod(temp_sec, 60)
        print('\n Time taken: %i hours %i minutes and %s seconds' % (thour, tmin, round(tsec, 2)))


@jit
def gini(y_true, y_prob):
    y_true = np.asarray(y_true)
    y_true = y_true[np.argsort(y_prob)]
    ntrue = 0
    gini = 0
    delta = 0
    n = len(y_true)
    for i in range(n - 1, -1, -1):
        y_i = y_true[i]
        ntrue += y_i
        gini += y_i * delta
        delta += 1 - y_i
    gini = 1 - 2 * gini / (ntrue * (n - ntrue))
    return gini


def evalerror(preds, dtrain):
    labels = dtrain.get_label()
    return 'gini', gini(labels, preds), True


def dropmissingcol(pdData):
    vars_to_drop = ['ps_car_03_cat', 'ps_car_05_cat']
    pdData.drop(vars_to_drop, inplace=True, axis=1)
    return pdData


def missingvalues(pdData):
    mean_imp = SimpleImputer(missing_values=-1, strategy='mean')
    mode_imp = SimpleImputer(missing_values=-1, strategy='most_frequent')
    features = ['ps_reg_03', 'ps_car_12', 'ps_car_14', 'ps_car_11']
    for i in features:
        if i == 'ps_car_11':
            pdData[i] = mode_imp.fit_transform(pdData[[i]]).ravel()
        else:
            pdData[i] = mean_imp.fit_transform(pdData[[i]]).ravel()
    return pdData


def encodecat(train, test):
    cat_features = [col for col in train.columns if '_cat' in col]
    for column in cat_features:
        temp = pd.get_dummies(pd.Series(train[column]), prefix=column)
        train = pd.concat([train, temp], axis=1)
        train = train.drop([column], axis=1)

    for column in cat_features:
        temp = pd.get_dummies(pd.Series(test[column]), prefix=column)
        test = pd.concat([test, temp], axis=1)
        test = test.drop([column], axis=1)
    return train, test


def RescaleData(train, test):
    scaler = StandardScaler()
    scaler.fit_transform(train)
    scaler.fit_transform(test)
    return train, test


def DropCalcCol(train, test):
    col_to_drop = train.columns[train.columns.str.startswith('ps_calc_')]
    train = train.drop(col_to_drop, axis=1)
    test = test.drop(col_to_drop, axis=1)
    return train, test

We begin by loading our data - This is the data where the missing values have already been imputed by a linear regression model.
We can later set our impute boolean to False and compare how effective this model was in comparison to a simple mean/mode imputation.

Applying the functions above, we have an encoded, rescaled dataset with missing values imputed. The targets have been seperated as new dataframes.

In general, it is not necessary to scale or encode your data when using boosting algorithms. Boosting algorithms typically work by building a model based on a combination of many weak models, each of which is trained on a subset of the data. This means that boosting algorithms are less sensitive to the scale of the data than some other types of algorithms, such as support vector machines. However, it is unlikely to lower the performance of the algorithm and in some cases could result in increased performance. In this example, it has been done since the functions have already been written for our previous models. Here, lightgbm will be using gradient boosted decision trees which are scale invariant. [4] [5]

In [33]:
impute = True #true if we use linear regression to impute missing values - false uses mean/mode imputation
if impute == True: #our imputed training set can be calculated by running the datahandling section of the code
    train = pd.read_csv("Dataset/ImputeTrain.csv")
    test = pd.read_csv("Dataset/ImputeTest.csv")
else:
    train = pd.read_csv("Dataset/new_train.csv")
    test = pd.read_csv("Dataset/new_test.csv")
    test = dropmissingcol(test)
    train = dropmissingcol(train)
target_test = test['target'].values
test = test.drop(['target'], axis=1)

#code to clean up any remaining missing values with mean impute
train = missingvalues(train)
test = missingvalues(test)

#removing these columns from the dataframes and saving them seperately
y_train = train['target'].values
train_id = train['id'].values
X = train.drop(['target', 'id'], axis=1)
test_id = test['id']
X_test = test.drop(['id'], axis=1)

#encoding, rescaling and dropping calc columns
X, X_test = DropCalcCol(X, X_test)
X, X_test = encodecat(X, X_test)
X = pd.DataFrame(X)
X_test = pd.DataFrame(X_test)
X, X_test = RescaleData(X, X_test)

We are now ready to implement the model.

XGboost vs LightGBM

Originally we set out on the project to implement an XGboost algorithm. The reasoning behind this was due to the fact that they often perform extremely well in tasks similar to the one here, especially when looking at the Kaggle leaderboards (albeit prone to overfitting)
When implementing a boosting algorithm, a huge part of the success comes from hyperparameter optimisation. [6]
A method we have previously looked at is Grid Search CV for hyperparameter optimisation - effectively brute searching through a collection of potential hyperparameter combinations and returning the best result. An issue with this method is that the more precision you want, the more combinations and possibilities you will have to try. [7]

An XGboost algorithm simply fell short on this big dataset as it was going to take a long time to run a grid search.
Possibilities were to reduce the dataset size for the grid search - say run the grid search on 20% of the data,
Or research into LightGBM, an alternative boosting method with solid claims of being a lot more efficient in run time. [8]

When implementing LightGBM the difference was huge. We could maintain the same full dataset and search through a large amount of hyperparameter combinations to optimise, which resulted in a noticeable score increase in comparison to our xgboost algorithm with a small hyperparameter search grid.

Therefore, for this project, we found LightGBM to be a better choice of algorithm.

# Grid Search

In [34]:
OPTIMIZE_ROUNDS = False
LEARNING_RATE = 0.07
EARLY_STOPPING_ROUNDS = 50

#paramaters to search over
params = {
    'min_child_weight': [5, 10, 12, 15, 30, 50, 100, 150],
    'num_leaves': [4, 5, 8, 10, 15, 20, 30],
    'subsample': [0.2, 0.4, 0.6, 0.8],
    'drop_rate': [0.1, 0.3, 0.5, 0.7, 0.15, 0.2],
    'max_depth': [3, 4, 5, 7, 10, 12, 15, 20]
}
#classifier model
model = lgbm.LGBMClassifier(learning_rate=LEARNING_RATE, n_estimators=600, objective='binary', )

#folds to use in stratified k-fold
folds = 3
#how many combinations of the above parameters should we try
param_comb = 10
#the algorithm is going to run folds x param_comb times

SKfold = StratifiedKFold(n_splits=folds, shuffle=True, random_state=1)
#set up search with SKfold split
random_search = RandomizedSearchCV(model, param_distributions=params, n_iter=param_comb, scoring='roc_auc', n_jobs=4,
                                   cv=SKfold.split(X, y_train), verbose=3, random_state=1)


#UNCOMMENT THIS SECTION TO START GRID SEARCH - CODE COMPLETION ~4 MIN
'''
start_time = timer(None)
#start search
random_search.fit(X, y_train)
timer(start_time)

print('\n All results:')
print(random_search.cv_results_)
print('\n Best estimator:')
print(random_search.best_estimator_)
print('\n Best Normalised gini score for %d-fold search with %d parameter combinations:' % (folds, param_comb))
print(random_search.best_score_)
print('\n Best hyperparameters:')
print(random_search.best_params_)
results = pd.DataFrame(random_search.cv_results_)
results.to_csv('lightgbm-randomgridsearch-results-03.csv')
'''

"\nstart_time = timer(None)\n#start search\nrandom_search.fit(X, y_train)\ntimer(start_time)\n\nprint('\n All results:')\nprint(random_search.cv_results_)\nprint('\n Best estimator:')\nprint(random_search.best_estimator_)\nprint('\n Best Normalised gini score for %d-fold search with %d parameter combinations:' % (folds, param_comb))\nprint(random_search.best_score_)\nprint('\n Best hyperparameters:')\nprint(random_search.best_params_)\nresults = pd.DataFrame(random_search.cv_results_)\nresults.to_csv('lightgbm-randomgridsearch-results-03.csv')\n"

In grid search, a set of possible values for each hyperparameter is defined, and the combination of these values forms a grid. The grid search algorithm will then train the model with each combination of hyperparameters and evaluate the performance of the model on a validation set. The combination of hyperparameters that produces the best performance on the validation set is chosen as the best set of hyperparameters for the model.

Grid search can be computationally expensive, as it requires training the model multiple times with different combinations of hyperparameters. However, it is a simple and effective method for finding the best hyperparameters for a given model and dataset.

Here we have set up a grid search with the hyperparameters of interest to search over. For the remaining hyperparameters, we can make a good guess from LightGBM literature online or they lack importance to fine tune in this particular case.

In this notebook, we have set to 3 folds and 10 combinations only to make the notebook accessible.

We run the grid search over a k=5 Stratified k-fold and search through with n_iter as the amount of combinations we wish to look at.
This is still a time intensive exercise, it is run in parallel across 4 chains but the LightGBM model has to train on five folds for each combination, and this is about 20-30 seconds each. For the strongest parameters I will use in the final model, we searched 200 combinations taking around 2 hours. This is overkill

# Bayesian Optimisation

Our grid search experienced a lot of flaws regarding the brute search aspect. It just isn't feasible to try every hyperparameter combination possible to really find our best combination. This led us to try a bayesian optimisation method, where in a bayesian way, we start with vague priors with weight on the possible values of the hyperparameters and update that distribution as the search goes on. This allows us to focus around the 'optimum' value much more quickly and waste less time looking at bad combinations. Furthermore, we can find continuous values of our hyperparameters whereas grid search was limited to the values we put into the grid. [9]

Overall, this is a far superior method for our application here.

In [35]:
def evaluate_model(num_leaves, min_child_weight, feature_fraction, subsample, drop_rate, max_depth):
    params = {
        "objective": "binary",
        "boosting_type": "gbdt",
        "learning_rate": 0.07,
        "verbosity": -1,
        "num_leaves": int(num_leaves),
        "min_child_weight": min_child_weight,
        "feature_fraction": feature_fraction,
        "subsample": subsample,
        'drop_rate': drop_rate,
        'max_depth': int(max_depth)
    }
    num_boost_round = 10000

    # define the number of folds for cross-validation
    n_folds = 5

    # create a stratified k-fold iterator
    skf = StratifiedKFold(n_splits=n_folds, shuffle=True, random_state=1)

    # initialize a list to store the evaluation metric for each fold
    scores = []

    # iterate over the folds
    for id_train, id_val in skf.split(X, y_train):
        # get the training and validation data for this fold
        X_train_fold = X.iloc[id_train]
        y_train_fold = y_train[id_train]
        X_val_fold = X.iloc[id_val]
        y_val_fold = y_train[id_val]

        lgb_train = lgbm.Dataset(X_train_fold, y_train_fold)
        lgb_val = lgbm.Dataset(X_val_fold, y_val_fold)

        # train the model with the specified parameters on the training data
        model = lgbm.train(params, lgb_train, num_boost_round, valid_sets=lgb_val, feval=evalerror, verbose_eval=100, early_stopping_rounds=100)
        scores.append(model.best_score['valid_0']['gini'])

    # return the mean evaluation metric across all folds
    return np.mean(scores)

# define the hyperparameters to be optimised
hyperparameters = {
    "num_leaves": (4, 50),
    "min_child_weight": (0.001, 150),
    "feature_fraction": (0.1, 0.9),
    "subsample": (0.1, 1),
    'drop_rate': (0.1, 0.8),
    'max_depth': (3, 20)
}
#UNCOMMENT TO START BAYESIAN OPTIMISATION ~10 MINS
'''
# perform Bayesian optimisation to find the optimal hyperparameters
optimizer = BayesianOptimization(evaluate_model, hyperparameters)
optimizer.maximize(n_iter=10)

# display the optimal values of the hyperparameters
print("Optimal hyperparameters:")
print(optimizer.max)
'''

'\n# perform Bayesian optimisation to find the optimal hyperparameters\noptimizer = BayesianOptimization(evaluate_model, hyperparameters)\noptimizer.maximize(n_iter=10)\n\n# display the optimal values of the hyperparameters\nprint("Optimal hyperparameters:")\nprint(optimizer.max)\n'

In [36]:
'''
Best hyperparameters from grid search:
{'subsample': 0.2, 'num_leaves': 15, 'min_child_weight': 150, 'max_depth': 3, 'drop_rate': 0.15}
'''
'''
Optimal hyperparameters from bayesian optimisation:
{'target': 0.28495605847657257, 'params': {'drop_rate': 0.22059703601445746, 'feature_fraction': 0.5855988837603003, 'max_depth': 17.666370012570326, 'min_child_weight': 139.73583540367778, 'num_leaves': 19.340987296541584, 'subsample': 0.2208063179655893}}
'''

"\nOptimal hyperparameters from bayesian optimisation:\n{'target': 0.28495605847657257, 'params': {'drop_rate': 0.22059703601445746, 'feature_fraction': 0.5855988837603003, 'max_depth': 17.666370012570326, 'min_child_weight': 139.73583540367778, 'num_leaves': 19.340987296541584, 'subsample': 0.2208063179655893}}\n"

# LightGBM model

Now we have found our best parameters, we are ready to train the model and predict on the test set.

When working with a dataset that has a class imbalance, stratified k-fold can be especially useful. This is because a dataset with a class imbalance can cause the model to be biased towards the majority class, and the evaluation of the model may be misleading if the folds are not representative of the class distribution in the dataset. By using stratified k-fold cross-validation, the model can be trained and evaluated on balanced folds, which can provide more accurate estimates of the model's performance.

We also wish to acknowledge the issue of overfitting, so we set early stopping times and iterate this process of generating our predictions by averaging over different folds and furthermore averaging the whole process over different seeds. Taking only our best folds would overfit here. Using multiple models trained with different random seeds will have slightly different parameter values and will make slightly different predictions. Averaging the predictions of these models can help smooth out any overfitting that may have occurred in individual models,

In [37]:
#use our best parameters
'''
Optimal hyperparameters from bayesian optimisation:
{'target': 0.28495605847657257, 'params': {'drop_rate': 0.22059703601445746, 'feature_fraction': 0.5855988837603003, 'max_depth': 17.666370012570326, 'min_child_weight': 139.73583540367778, 'num_leaves': 19.340987296541584, 'subsample': 0.2208063179655893}}
'''

min_data_in_leaf = 2000
num_boost_round = 10000
params = {"objective": "binary",
          "boosting_type": "gbdt",
          "learning_rate": 0.07,
          "max_bin": 256,
          "verbosity": -1,
          "feature_fraction": 0.5855988837603003,
          'subsample': 0.220806,
          'num_leaves': 19,
          'min_child_weight': 140,
          'max_depth': 18,
          'drop_rate': 0.22059703601445746
          }

folds = 5

SKfold = StratifiedKFold(n_splits=folds, shuffle=True, random_state=1)

#empty, will save our scores here in the future
best_trees = []
fold_scores = []

#cv_train = np.zeros(len(y_train))
cv_pred = np.zeros(len(X_test))

start_time = timer(None)
#iterations each have a different seed, we average over these to prevent overfit
iterations = 3
for seed in range(iterations):
    timer(start_time)
    params['seed'] = seed
    #start SK fold
    for id_train, id_test in SKfold.split(X, y_train):
        #x train, x validation
        xtr, xvl = X.loc[id_train], X.loc[id_test]
        #y train, y validation
        ytr, yvl = y_train[id_train], y_train[id_test]
        #efficient datastructures for lgbm
        dtrain = lgbm.Dataset(data=xtr, label=ytr)
        dval = lgbm.Dataset(data=xvl, label=yvl, reference=dtrain)
        #model training
        bst = lgbm.train(params, dtrain, num_boost_round, valid_sets=dval, feval=evalerror, verbose_eval=100,
                         early_stopping_rounds=100)
        #add best tree and fold scores to data structure
        best_trees.append(bst.best_iteration)
        fold_scores.append(bst.best_score)
        #predict for our test set with best tree from fold. Sums the probabilities
        cv_pred += bst.predict(X_test, num_iteration=bst.best_iteration)
    #average the predictions for our 5 folds

pd.DataFrame({'id': test_id, 'target': cv_pred / (iterations * folds)}).to_csv('lgbm_bayesian_opt.csv', index=False)

if Kaggle == False:
    test_score = gini(target_test, cv_pred / (iterations * folds))
    print("Score on the test data")
    print("Gini")
    print(test_score)


 Time taken: 0 hours 0 minutes and 0.0 seconds
Training until validation scores don't improve for 100 rounds
[100]	valid_0's binary_logloss: 0.151177	valid_0's gini: 0.286179
[200]	valid_0's binary_logloss: 0.15118	valid_0's gini: 0.286243
Early stopping, best iteration is:
[133]	valid_0's binary_logloss: 0.151113	valid_0's gini: 0.287715
Training until validation scores don't improve for 100 rounds
[100]	valid_0's binary_logloss: 0.150902	valid_0's gini: 0.294898
[200]	valid_0's binary_logloss: 0.150811	valid_0's gini: 0.296848
[300]	valid_0's binary_logloss: 0.150866	valid_0's gini: 0.296176
Early stopping, best iteration is:
[212]	valid_0's binary_logloss: 0.150801	valid_0's gini: 0.297337
Training until validation scores don't improve for 100 rounds
[100]	valid_0's binary_logloss: 0.15189	valid_0's gini: 0.267312
[200]	valid_0's binary_logloss: 0.151864	valid_0's gini: 0.269072
Early stopping, best iteration is:
[168]	valid_0's binary_logloss: 0.151828	valid_0's gini: 0.269801
Tra

Our final result on the test set is

In [38]:
'GRID SEARCH PARAMETERS'
"Score on the test data"
"Gini"
"0.2788912296158311"

'BAYESIAN OPTIMISATION PARAMETERS'
"Score on the test data"
"Gini"
"0.2817052971888222"

'0.2817052971888222'

Separately, we run the same code with the mean imputed data and gridsearch parameters and our score is 0.2704817256788792

# References

[1] https://ieeexplore.ieee.org/stamp/stamp.jsp?tp=&arnumber=4809227
Vaghela, Ganatra, Thakkar - Boost a Weak Learner to a Strong Learner Using Ensemble System Approach - 2009 - DOI:10.1109/IADCC.2009.4809227
[2] https://www.toptal.com/machine-learning/ensemble-methods-kaggle-machine-learn
[3] https://lightgbm.readthedocs.io/en/latest/Features.html#optimization-in-speed-and-memory-usage
[4] https://sebastianraschka.com/faq/docs/when-to-standardize.html
[5] https://en.wikipedia.org/wiki/Decision_tree_learning#Advantages
[6] https://towardsdatascience.com/understanding-hyperparameters-and-its-optimisation-techniques-f0debba07568
[7] https://web.archive.org/web/20160701182750/http://blog.dato.com/how-to-evaluate-machine-learning-models-part-4-hyperparameter-tuning
[8] https://www.geeksforgeeks.org/lightgbm-vs-xgboost-which-algorithm-is-better/
[9] http://neupy.com/2016/12/17/hyperparameter_optimization_for_neural_networks.html







